In [15]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.path as mplPath
import shapefile
from netCDF4 import Dataset
import dateutil.parser
from shapely.geometry import Polygon, Point

In [16]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"

In [17]:
data_df = gpd.read_file(shapefile_name)

In [18]:
numLayers = 5;
numSites = data_df.shape[0]
numTargetSites = numSites

In [5]:
boxes = [75, 78, 83, 48, 55, 12];
boxID = 11;

In [19]:
startCountDay = -1;
desiredDT = 12;

numStepsPerDT = 12;
numStepsPerDT = 1;

outputDT = 86400
outputDT = 3640
outputDT = 21600    # 6 hours.
outputDT = 60*60

In [20]:
stepsPerDay = int(86400.0/ outputDT);
numStepsPerDT = stepsPerDay;
numStepsPerDT = int(outputDT/3600.0)

debug = False

In [21]:
# Define paths
paths = {
    'out': './results',
}


In [30]:
outFileName = os.path.join(paths['out'], 'EddyParticles.nc')

try:
        pfile = xr.open_dataset(str(outFileName), decode_cf=True)
except:
        pfile = xr.open_dataset(str(outFileName), decode_cf-False)

SyntaxError: keyword can't be an expression (<ipython-input-30-9a1fa529620e>, line 4)

In [27]:
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

AttributeError: 'str' object has no attribute 'variables'

In [15]:
numParticles = lon.shape[0]

trackDates = [];

for i in range(0,numParticles):
    print(time[i][0])
    #trackDates.append( dateutil.parser.parse(time[i][0]))
    trackDates.append(time[i][0])

NameError: name 'lon' is not defined

In [18]:
RDiff = max(trackDates) - min(trackDates)

minDate = np.datetime64("2018-01-01T00:30:00");
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

NameError: name 'trackDates' is not defined

In [19]:
numReleaseDays = 1;
numReleaseSteps = numReleaseDays * stepsPerDay;
trackLength = len(lon[0]);

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))

numSteps = int(trackLength / numStepsPerDT)

NameError: name 'lon' is not defined

In [21]:
# Create the netcdf

netcdfFileName = "Wind_Oil_Box_" + str(boxID) + "_dt_" + str(outputDT) + ".nc"

try:
    os.remove(netcdfFileName)
except:
    pass

ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)

time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

NameError: name 'netcdfFileName' is not defined

In [22]:
times = ncfile.createVariable("time","f4",("t",))
times.units = "seconds since " + d

times.dt = str(outputDT);
oil = ncfile.createVariable("Oil","f4",("t", "b",))

timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData;

boxDispersal = np.zeros((numSteps + numReleaseSteps, numTargetSites));


NameError: name 'ncfile' is not defined

In [ ]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate;
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)));

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT];
        partLat = lat[partIndex][stepIndex * numStepsPerDT];
        partProb = probs[partIndex][stepIndex * numStepsPerDT];

        matchFound = 0;

        for targetIndex in range(0, numTargetSites):

            path = data_df.iloc[targetIndex].geometry
            checks = path.contains(Point(partLon, partLat));

            if checks:
                boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + partProb;

                # uncomment line below to ignore particle decay during debugging.
                #boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + 1.0

                matchFound = 1
                if debug:
                    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))


                break;

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)


        #break

oil[:, :] = boxDispersal;

ncfile.close()